# Overview

Terrain is the web API that the CyVese Discovery Environment uses to perform all of its tasks. Terrain is a REST-like web API. That is, it acts like a traditional web server in many ways. Requests that are sent to terrain are formatted just like a request that your browser might make in order to open a web page. Responses from Terrain are typically very similar to responses that your browser might receive from a request. The primary difference is the response format. The format that your browser receives from a web server will typically be formatted as a a web page whereas a response from Terrain will usually be a document formatted in JavaScript Object Notation (JSON). The primary reason for this difference is that JSON is easier for computers to read than HTML, and it's easier for humans to read than another common format in use on the internet, XML. If you're unfamiliar with JSON, DigitalOcean has this great [introduction](https://www.digitalocean.com/community/tutorials/an-introduction-to-json) that will quickly bring you up to speed.

Without further ado, let's give Terrain a try! 

## Prerequisites

Before we can use Terrain, we'll need some prerequisites. The `requests` library is an easy way to make requests to web services in Python, so let's import it so that we can use it later. It'll also be necessary to prompt for a password and to pretty-print some data structures.

In [ ]:
import getpass
import pprint
import requests

## The Welcome Message

Terrain has a bit of a goofy welcome message that you can see by hitting its root endpoint. The original purpose of the message was for developers to have an easy way to verify that Terrain was in fact running and responsive. We still use this endpoint on a regular basis, but now calls to it are now automated. For some reason, the computers don't find it as amusing as the humans do.

In [ ]:
r = requests.get("https://de.cyverse.org/terrain/")
r.raise_for_status()
print(r.text)

## Authentication

To do anything interesting in terrain, you have to authenticate to the service. Terrain uses Apereo's [Central Authentication Service](https://apereo.github.io/cas/5.3.x/) OAuth provider for authentication. Typically you'd need to regsiter a client to allow you to authenticate directly to CAS, which can be a lot of work. For this reason, Terrain has an endpoint that you can use to obtain a token indirectly using Terrain's own OAuth client. But in order to get the access token, we'll need your password.

In [ ]:
print("Username: ", end='', flush=True)
username = input()
print("Password: ", end='', flush=True)
password = getpass.getpass()

And now we can obtain an access token.

In [ ]:
r = requests.get("https://de.cyverse.org/terrain/token", auth=(username, password))
r.raise_for_status()
token = r.json()['access_token']
auth_headers = {"Authorization": "Bearer " + token}

## Searching for Apps

_Now_ we can do something interesting. Suppose you've got an eight-hundred-word essay to write, and you want to use the DE to make sure that your essay is long enough. You can use one of the word count endpoints to find out!

In [ ]:
query_params = {"search": "DE Word Count"}
r = requests.get("https://de.cyverse.org/terrain/apps", headers=auth_headers, params=query_params)
r.raise_for_status()
pprint.pprint(r.json())